In [1]:
import pandas as pd
train = pd.read_csv('package-lock.csv')
test = pd.read_csv('test_tweets_anuFYb8.csv')

train = train.reindex(['id','tweet','label'], axis=1)
train.sample(3)

,id,tweet,label
30419,30420,very #little is needed to make a life; it is...,0
25322,25323,uhhhh how long has it been since we graduated ...,0
31590,31591,my girl said she never had a nigga to wear l...,0


In [2]:
test.sample(3)

,id,tweet
4175,36138,#monday everyone! #wishing u a #magical week...
8678,40641,@user #alimaong is 7 years old today #bihda...
3677,35640,@user new episode of @user &amp; 4th show of ...


In [3]:
print(train.shape, test.shape)

(31962, 3) (17197, 2)


In [4]:
X = train.iloc[:, :-1].values
y = train.iloc[:, 2].values

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

print("X_train: " + str(X_train) + "\n")
print("X_test: " + str(X_test) + "\n")
print("y_train: " + str(y_train) + "\n")
print("y_test: " + str(y_test) + "\n")


X_train: [[13487
  'how could you all ever lose your faith in me....   #pain #mistrust #relationship #outcast  #twitter #life #love #friendship #cruel']
 [3936 'my name often times auto corrects to leukemia  ']
 [26592
  "suddenly staing to feel real now i'm finished both jobs ð\x9f\x98»  "]
 ...
 [9846
  'another #melbourne snap, this guy played the most beautiful sounding instrument!   #streetphotography ']
 [10800 '@user thanks for the retweet :)  ']
 [2733
  " @user .@user kicks off today! check out the full list of guests we're   to see this weekend! "]]

X_test: [[13668 'i am thankful for sunshine.#thankful #positive   ']
 [22091
  "up late....i am tired but i can't sleep. my eyes are swollen from crying. my brothers and sisters ð\x9f\x91\xadð\x9f\x91¬ð\x9f\x8c\x88ð\x9f\x91¼ð\x9f\x99\x8fð\x9f\x98¢   #prayfoheworld #pulse #help"]
 [21398 'series finale of house of lies tonight.  ']
 ...
 [20628
  ' @user pls  #norfolkhour the eaaa norfolk polo festival stas tomorrow!  !  ']
 [1558

In [5]:
from nltk.tokenize import TweetTokenizer

tokenizer = TweetTokenizer()

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(ngram_range=(1, 2), tokenizer=tokenizer.tokenize)

In [7]:
vectorizer

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=None,
                min_df=1, ngram_range=(1, 2), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=<bound method TweetTokenizer.tokenize of <nltk.tokenize.casual.TweetTokenizer object at 0x10e0d85c0>>,
                use_idf=True, vocabulary=None)

In [8]:
full_text = list(train['tweet'].values) + list(test['tweet'].values)
vectorizer.fit(full_text)
train_vectorized = vectorizer.transform(train['tweet'])
test_vectorized = vectorizer.transform(test['tweet'])

In [9]:
%%time
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
logreg = LogisticRegression()
ovr = OneVsRestClassifier(logreg)

ovr.fit(train_vectorized, y)

/Users/saurabh/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


CPU times: user 1.47 s, sys: 63 ms, total: 1.53 s
Wall time: 1.05 s


OneVsRestClassifier(estimator=LogisticRegression(C=1.0, class_weight=None,
                                                 dual=False, fit_intercept=True,
                                                 intercept_scaling=1,
                                                 l1_ratio=None, max_iter=100,
                                                 multi_class='warn',
                                                 n_jobs=None, penalty='l2',
                                                 random_state=None,
                                                 solver='warn', tol=0.0001,
                                                 verbose=0, warm_start=False),
                    n_jobs=None)

In [10]:
import numpy as np
from sklearn.model_selection import cross_val_score
scores = cross_val_score(ovr, train_vectorized, y, scoring='accuracy', n_jobs=-1, cv=3)
print('Cross-validation mean accuracy {0:.2f}%, std {1:.2f}.'.format(np.mean(scores) * 100, np.std(scores) * 100))

Cross-validation mean accuracy 94.01%, std 0.06.


In [11]:
%%time
from sklearn.svm import LinearSVC
svc = LinearSVC(dual=False)
scores = cross_val_score(svc, train_vectorized, y, scoring='accuracy', n_jobs=-1, cv=3)
print('Cross-validation mean accuracy {0:.2f}%, std {1:.2f}.'.format(np.mean(scores) * 100, np.std(scores) * 100))

Cross-validation mean accuracy 95.77%, std 0.10.
CPU times: user 35.9 ms, sys: 25.6 ms, total: 61.5 ms
Wall time: 3.37 s


In [12]:
ovr.fit(train_vectorized, y);
svc.fit(train_vectorized, y);

/Users/saurabh/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [13]:
predictions = svc.predict(test_vectorized)

In [14]:
test['label'] = predictions
test.sample(3)

,id,tweet,label
795,32758,@user @user @user @user right.just like presid...,0
10474,42437,@user yay finally got it!!! special thanks to...,0
14309,46272,when i feel is a children's story about reco...,0


In [15]:
test.to_csv('submission.csv', index=False)

'










## Deep Learning

In [16]:
def format_data(train, test, max_features, maxlen):

    from keras.preprocessing.text import Tokenizer
    from keras.preprocessing.sequence import pad_sequences
    from keras.utils import to_categorical
    
    train = train.sample(frac=1).reset_index(drop=True)
    train['tweet'] = train['tweet'].apply(lambda x: x.lower())
    test['tweet'] = test['tweet'].apply(lambda x: x.lower())

    X = train['tweet']
    test_X = test['tweet']
    Y = to_categorical(train['label'].values)

    tokenizer = Tokenizer(num_words=max_features)
    tokenizer.fit_on_texts(list(X))

    X = tokenizer.texts_to_sequences(X)
    X = pad_sequences(X, maxlen=maxlen)
    test_X = tokenizer.texts_to_sequences(test_X)
    test_X = pad_sequences(test_X, maxlen=maxlen)

    return X, Y, test_X

In [18]:
maxlen = 125
max_features = 10000

X, Y, test_X = format_data(train, test, max_features, maxlen)

Using TensorFlow backend.


In [19]:
seed = 0
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.25, random_state=seed)


In [20]:
from keras.layers import Input, Dense, Embedding, Flatten
from keras.layers import SpatialDropout1D
from keras.layers.convolutional import Conv1D, MaxPooling1D
from keras.models import Sequential
model = Sequential()

# Input / Embdedding
model.add(Embedding(max_features, 150, input_length=maxlen))

# CNN
model.add(SpatialDropout1D(0.2))

model.add(Conv1D(32, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))

model.add(Conv1D(64, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))

model.add(Flatten())

# Output layer
model.add(Dense(2, activation='sigmoid'))

W0814 16:51:45.661587 4631000512 deprecation_wrapper.py:119] From /Users/saurabh/anaconda3/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0814 16:51:45.773864 4631000512 deprecation_wrapper.py:119] From /Users/saurabh/anaconda3/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0814 16:51:45.798799 4631000512 deprecation_wrapper.py:119] From /Users/saurabh/anaconda3/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0814 16:51:45.876634 4631000512 deprecation_wrapper.py:119] From /Users/saurabh/anaconda3/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_defa

In [21]:
epochs = 5
batch_size = 32

In [22]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, Y_train, validation_data=(X_val, Y_val), epochs=epochs, batch_size=batch_size, verbose=1)

W0814 16:51:46.279360 4631000512 deprecation_wrapper.py:119] From /Users/saurabh/anaconda3/lib/python3.6/site-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0814 16:51:46.374469 4631000512 deprecation_wrapper.py:119] From /Users/saurabh/anaconda3/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:3295: The name tf.log is deprecated. Please use tf.math.log instead.

W0814 16:51:46.632503 4631000512 deprecation.py:323] From /Users/saurabh/anaconda3/lib/python3.6/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 23971 samples, validate on 7991 samples
Epoch 1/5
23971/23971 [==============================] - 125s 5ms/step - loss: 0.1674 - acc: 0.9463 - val_loss: 0.1149 - val_acc: 0.9595
Epoch 2/5
23971/23971 [==============================] - 130s 5ms/step - loss: 0.0729 - acc: 0.9741 - val_loss: 0.1117 - val_acc: 0.9638
Epoch 3/5
23971/23971 [==============================] - 118s 5ms/step - loss: 0.0329 - acc: 0.9885 - val_loss: 0.1377 - val_acc: 0.9641
Epoch 4/5
23971/23971 [==============================] - 120s 5ms/step - loss: 0.0126 - acc: 0.9959 - val_loss: 0.1896 - val_acc: 0.9637
Epoch 5/5
23971/23971 [==============================] - 104s 4ms/step - loss: 0.0049 - acc: 0.9986 - val_loss: 0.2235 - val_acc: 0.9583


In [23]:
test['label'] = model.predict_classes(test_X, batch_size=batch_size, verbose=1)
test.to_csv('sub_cnn.csv', index=False)

17197/17197 [==============================] - 11s 628us/step


# Analytics Vidhya
- https://www.analyticsvidhya.com/blog/2018/04/a-comprehensive-guide-to-understand-and-implement-text-classification-in-python/

Getting your machine ready
Lets implement basic components in a step by step manner in order to create a text classification framework in python. To start with, import all the required libraries.

You would need requisite libraries to run this code – you can install them at their individual official links

- Pandas
- Scikit-learn
- XGBoost
= TextBlob
- Keras


In [ ]:
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble

import pandas as pd, numpy as np
import xgboost, textblob, string
from keras.preprocessing import text, sequence
from keras import layers, models, optimizers

## 1. Dataset preparation
For the purpose of this article, I am the using dataset of amazon reviews which can be downloaded at this [link](https://gist.github.com/kunalj101/ad1d9c58d338e20d09ff26bcc06c4235). The dataset consists of 3.6M text reviews and their labels, we will use only a small fraction of data. To prepare the dataset, load the downloaded data into a pandas dataframe containing two columns – text and label. (Source)

In [ ]:
# load the dataset
data = open('data/corpus').read()
labels, texts = [], []
for i, line in enumerate(data.split("\n")):
    content = line.split()
    labels.append(content[0])
    texts.append(" ".join(content[1:]))

# create a dataframe using texts and lables
trainDF = pd.DataFrame()
trainDF['text'] = texts
trainDF['label'] = labels

In [ ]:
trainDF.sample(3)

Next, we will split the dataset into training and validation sets so that we can train and test classifier. Also, we will encode our target column so that it can be used in machine learning models.

In [ ]:
# split the dataset into training and validation datasets 
train_x, valid_x, train_y, valid_y = model_selection.train_test_split(trainDF['text'], trainDF['label'])

# label encode the target variable 
encoder = preprocessing.LabelEncoder()
train_y = encoder.fit_transform(train_y)
valid_y = encoder.fit_transform(valid_y)

## 2. Feature Engineering
The next step is the feature engineering step. In this step, raw text data will be transformed into feature vectors and new features will be created using the existing dataset. We will implement the following different ideas in order to obtain relevant features from our dataset.

2.1 Count Vectors as features
2.2 TF-IDF Vectors as features

Word level
N-Gram level
Character level
2.3 Word Embeddings as features
2.4 Text / NLP based features
2.5 Topic Models as features

Lets look at the implementation of these ideas in detail.


### 2.1 Count Vectors as features
Count Vector is a matrix notation of the dataset in which every row represents a document from the corpus, every column represents a term from the corpus, and every cell represents the frequency count of a particular term in a particular document.

In [ ]:
# create a count vectorizer object 
count_vect = CountVectorizer(analyzer='word', token_pattern=r'\w{1,}')
count_vect.fit(trainDF['text'])

# transform the training and validation data using count vectorizer object
xtrain_count =  count_vect.transform(train_x)
xvalid_count =  count_vect.transform(valid_x)

In [ ]:
count_vect

## 2.2 TF-IDF Vectors as features
TF-IDF score represents the relative importance of a term in the document and the entire corpus. TF-IDF score is composed by two terms: the first computes the normalized Term Frequency (TF), the second term is the Inverse Document Frequency (IDF), computed as the logarithm of the number of the documents in the corpus divided by the number of documents where the specific term appears.

TF(t) = (Number of times term t appears in a document) / (Total number of terms in the document)
IDF(t) = log_e(Total number of documents / Number of documents with term t in it)

TF-IDF Vectors can be generated at different levels of input tokens (words, characters, n-grams)

a. **Word Level TF-IDF** : Matrix representing tf-idf scores of every term in different documents  
b. **N-gram Level TF-IDF** : N-grams are the combination of N terms together. This Matrix representing tf-idf scores of N-grams  
c. **Character Level TF-IDF** : Matrix representing tf-idf scores of character level n-grams in the corpus

In [ ]:
# word level tf-idf
tfidf_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=5000)
tfidf_vect.fit(trainDF['text'])
xtrain_tfidf =  tfidf_vect.transform(train_x)
xvalid_tfidf =  tfidf_vect.transform(valid_x)

# ngram level tf-idf 
tfidf_vect_ngram = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', ngram_range=(2,3), max_features=5000)
tfidf_vect_ngram.fit(trainDF['text'])
xtrain_tfidf_ngram =  tfidf_vect_ngram.transform(train_x)
xvalid_tfidf_ngram =  tfidf_vect_ngram.transform(valid_x)

# characters level tf-idf
tfidf_vect_ngram_chars = TfidfVectorizer(analyzer='char', token_pattern=r'\w{1,}', ngram_range=(2,3), max_features=5000)
tfidf_vect_ngram_chars.fit(trainDF['text'])
xtrain_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(train_x) 
xvalid_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(valid_x) 

## 2.3 Word Embeddings
A word embedding is a form of representing words and documents using a dense vector representation. The position of a word within the vector space is learned from text and is based on the words that surround the word when it is used. Word embeddings can be trained using the input corpus itself or can be generated using pre-trained word embeddings such as Glove, FastText, and Word2Vec. Any one of them can be downloaded and used as transfer learning. One can read more about word embeddings [here](https://www.analyticsvidhya.com/blog/2017/06/word-embeddings-count-word2veec/).

Following snnipet shows how to use pre-trained word embeddings in the model. There are four essential steps:

1. Loading the pretrained word embeddings
2. Creating a tokenizer object
3. Transforming text documents to sequence of tokens and pad them
4. Create a mapping of token and their respective embeddings  

You can download the pre-trained word embeddings from [here](https://s3-us-west-1.amazonaws.com/fasttext-vectors/wiki-news-300d-1M.vec.zip)

In [ ]:
# load the pre-trained word-embedding vectors 
embeddings_index = {}
for i, line in enumerate(open('data/wiki-news-300d-1M.vec')):
    values = line.split()
    embeddings_index[values[0]] = np.asarray(values[1:], dtype='float32')

# create a tokenizer 
token = text.Tokenizer()
token.fit_on_texts(trainDF['text'])
word_index = token.word_index

# convert text to sequence of tokens and pad them to ensure equal length vectors 
train_seq_x = sequence.pad_sequences(token.texts_to_sequences(train_x), maxlen=70)
valid_seq_x = sequence.pad_sequences(token.texts_to_sequences(valid_x), maxlen=70)

# create token-embedding mapping
embedding_matrix = np.zeros((len(word_index) + 1, 300))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

# 2.4 Text / NLP based features
A number of extra text based features can also be created which sometimes are helpful for improving text classification models. Some examples are:

1. Word Count of the documents – total number of words in the documents
2. Character Count of the documents – total number of characters in the documents
3. Average Word Density of the documents – average length of the words used in the documents
4. Puncutation Count in the Complete Essay – total number of punctuation marks in the documents
5. Upper Case Count in the Complete Essay – total number of upper count words in the documents
6. Title Word Count in the Complete Essay – total number of proper case (title) words in the documents
7. Frequency distribution of Part of Speech Tags:
  - Noun Count
  - Verb Count
  - Adjective Count
  - Adverb Count
  - Pronoun Count  

These features are highly experimental ones and should be used according to the problem statement only.

In [ ]:
trainDF['char_count'] = trainDF['text'].apply(len)
trainDF['word_count'] = trainDF['text'].apply(lambda x: len(x.split()))
trainDF['word_density'] = trainDF['char_count'] / (trainDF['word_count']+1)
trainDF['punctuation_count'] = trainDF['text'].apply(lambda x: len("".join(_ for _ in x if _ in string.punctuation))) 
trainDF['title_word_count'] = trainDF['text'].apply(lambda x: len([wrd for wrd in x.split() if wrd.istitle()]))
trainDF['upper_case_word_count'] = trainDF['text'].apply(lambda x: len([wrd for wrd in x.split() if wrd.isupper()]))

In [ ]:
pos_family = {
    'noun' : ['NN','NNS','NNP','NNPS'],
    'pron' : ['PRP','PRP$','WP','WP$'],
    'verb' : ['VB','VBD','VBG','VBN','VBP','VBZ'],
    'adj' :  ['JJ','JJR','JJS'],
    'adv' : ['RB','RBR','RBS','WRB']
}

# function to check and get the part of speech tag count of a words in a given sentence
def check_pos_tag(x, flag):
    cnt = 0
    try:
        wiki = textblob.TextBlob(x)
        for tup in wiki.tags:
            ppo = list(tup)[1]
            if ppo in pos_family[flag]:
                cnt += 1
    except:
        pass
    return cnt

trainDF['noun_count'] = trainDF['text'].apply(lambda x: check_pos_tag(x, 'noun'))
trainDF['verb_count'] = trainDF['text'].apply(lambda x: check_pos_tag(x, 'verb'))
trainDF['adj_count'] = trainDF['text'].apply(lambda x: check_pos_tag(x, 'adj'))
trainDF['adv_count'] = trainDF['text'].apply(lambda x: check_pos_tag(x, 'adv'))
trainDF['pron_count'] = trainDF['text'].apply(lambda x: check_pos_tag(x, 'pron'))